# 自然语言处理 & 另类数据模块

本部分演示了如何使用自然语言处理模型 [BERT](https://arxiv.org/abs/1810.04805) 来进行文本编码。

编码之后，我们可以使用 `krtrader/models/mlp.py` 中定义的神经网络来进行前向传播。

In [ ]:
import sys
sys.path.append("..")

## 注意

- 由于 BERT 模型较大，所以下面导入 `BertModel` 的代码可能需要较长时间下载至本地。
- 关于 `MLP` 模型的超参数，用户可以自行修改网络层数、向量维度、激活函数等，定义中的版本是一个简单的 demo ，使用了三层网络和 `ReLU()` 激活函数。

In [ ]:
from models.bert import BertModel
from models.mlp import MLP

## 模型定义与编码

可以看到，语义相似的语句经过编码之后，其向量的余弦相似度会更高，表示它们更加相似。

In [ ]:
bert = BertModel()
mlp = MLP()

sentence1 = "Hello, my dog is cute"
sentence2 = "Hello, my cat is cute"
sentence3 = "Hello, my dog is not cute"
print(f"sentence1: {sentence1}")
print(f"sentence2: {sentence2}")
print(f"sentence3: {sentence3}")
print(f"Similarity of sentence1 and sentence2: {bert.encode(sentence1).dot(bert.encode(sentence2).T)}")
print(f"Similarity of sentence1 and sentence3: {bert.encode(sentence1).dot(bert.encode(sentence3).T)}")

用神经网络对编码后的文本进行前向传播，可以根据个人需要与现有数据集进行训练。

In [ ]:
print(f"Dimension of the Bert embeddings: {bert.encode(sentence1).shape}")
print("-----Affine transformation-----")
print(f"Dimension of the projected Bert embeddings: {mlp(bert.encode(sentence1)).shape}")

## 另类数据爬取

注意：以下脚本需要在 Chrome 内核中运行。

下面我们从[雅虎财经新闻](https://finance.yahoo.com/news/)网页中爬取新闻标题。

您可以将爬取的数据（如下方代码中的 `data` 变量）以 `csv`，`txt` 或 `json` 格式自行保存在 `krtrader/strategies/cache/` 目录下。

In [ ]:
from data.crawl import get_data, get_driver
from selenium.webdriver.common.by import By

url = "https://finance.yahoo.com/news/" # yahoo financial news, VPN needed
driver = get_driver(url)
data = get_data(driver, "TAG_NAME", "h3") # news titles
# print(data[0].text)
with open("../strategies/cache/news0.txt", "w") as f:
    for i in data:
        f.write(i.text)
        f.write("\n")

driver.close()